In [ ]:
!pip install -U dspy-ai

In [2]:
import dspy

In [6]:
!ollama run mistral:latest

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 232 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 232 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 232 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 232 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 232 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 232 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 232 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 232 MB/4.1 GB                  pulling manifest 
pulling e8a35b5937a5...   6% ▕                ▏ 233 MB/4.1 GB            

In [12]:
!ollama run phi:latest

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 04778965089b...   0% ▕                ▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕                ▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕                ▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕                ▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕                ▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕                ▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕                ▏    0 B/1.6 GB                  pulling manifest 

# Language Model (LM)

In [3]:
import dspy
phi_ollama = dspy.OllamaLocal(model='phi')
# its very important to configure dspy with the LM
dspy.settings.configure(lm=phi_ollama, max_tokens=1024)

# Signature 
* An example to classify sentiment of a given sentence
* An example to summarise a given passage / sentence

In [4]:
# sentiment classificaiton
sentence = "what is an LLM?"
classify = dspy.Predict('sentence -> sentiment')
classify(sentence=sentence)

Prediction(
    sentiment='Sentence: What is an LLM?\nSentiment: Neutral'
)

In [5]:
# Another example to summarize a given passage / sentence
document = """The 21-year-old made seven appearances for the Hammers and netted his only goal for them in a Europa League qualification round match against Andorran side FC Lustrains last season. Lee had two loan spells in League One last term, with Blackpool and then Colchester United. He scored twice for the U's but was unable to save them from relegation. The length of Lee's contract with the promoted Tykes has not been revealed. Find all the latest football transfers on our dedicated page."""
summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)
print(response.summary)

A 21-year-old player made seven appearances for the Hammers and scored one goal in a Europa League qualification round match against FC Lustrains last season. He had two loan spells in League One, scoring twice for Colchester United but unable to save them from


In [9]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""
    sentence = dspy.InputField()
    sentiment = dspy.OutputField()

sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='Sentence: I started feeling a little vulnerable when the giant spotlight started blinding me\nSentiment: Sadness'
)

# Modules

In [10]:
# 1) Declare with a signature.
classify = dspy.Predict('sentence -> sentiment')

# 2) Call with input argument(s). 
sentence = "it's a charming and often affecting journey."
response = classify(sentence=sentence)

# 3) Access the output.
print(response.sentiment)

Sentiment: Positive


In [11]:
question = "What's something great about the ColBERT retrieval model?"

# 1) Declare with a signature, and pass some config.
classify = dspy.ChainOfThought('question -> answer', n=3)

# 2) Call with input argument.
response = classify(question=question)

# 3) Access the outputs.
print(response)

Prediction(
    rationale='${produce_the_answer()}',
    answer='The ColBERT retrieval model is a significant improvement over BERT, as it was specifically designed for answering questions. It utilizes an encoder-decoder architecture and has been shown to achieve state-of-the-art results on various NLP tasks.',
    completions=Completions(...)
) (1 completions omitted)


# Data 
* Three values: the inputs, the intermediate labels, and the final label.
* 10 - 500 samples. More is better

In [12]:
train_set = [dspy.Example(question="Alice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?",
                          answer="The name of the third daughter is Alice."
                         ),
             dspy.Example(question="Why can camels survive for long without water?",
                          answer="Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time."
                         )]

# Metrics

In [13]:
def validate_answer(example, pred):
    return example.answer.lower() == pred.answer.lower()

# Optimizers

In [14]:
class qa_module(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
    
    def forward(self, question):
        prediction = self.generate_answer(question=question)
        return dspy.Prediction(answer=prediction.answer)

# A multi-hop QA Example (from DsPy docs)
We will do the below steps:
1. Load the Language Model and Retrieval Model
2. Load the "question-answer pairs" `HotPotQA` dataset to compile (train) the DsPy program
3. Build the signatures
4. Define the pipeline (as a module)
5. Define the evaluation metric
6. Compile (train) the pipeline with an optimizer (lets use `BootstrapFewShot`)
7. Evaluate and compare the Compiled and Uncompiled pipeline

## Load the LM and RM models

In [3]:
import dspy
phi_ollama = dspy.OllamaLocal(model='phi')

In [4]:
colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=phi_ollama, rm=colbertv2_wiki17_abstracts, max_tokens=1024)

## Load the Dataset 

In [40]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, 
                   train_size=20, 
                   eval_seed=2023, 
                   dev_size=10, 
                   test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

(20, 10)

## Build Signature

In [22]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

In [23]:
class GenerateSearchQuery(dspy.Signature):
    """Write a simple search query that will help answer a complex question."""
    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    query = dspy.OutputField()

## Define the pipeline

In [27]:
from dsp.utils import deduplicate

class SimplifiedBaleen(dspy.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()
        self.generate_query = [dspy.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = dspy.Retrieve(k=passages_per_hop)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops
    
    def forward(self, question):
        context = []
        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        pred = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=pred.answer)

## Define the evaluation metric 

In [6]:
def validate_context_and_answer_and_hops(example, pred, trace=None):
    # The predicted answer matches the gold answer.
    if not dspy.evaluate.answer_exact_match(example, pred): 
        return False
    # The retrieved context contains the gold answer.
    if not dspy.evaluate.answer_passage_match(example, pred): 
        return False

    hops = [example.question] + [outputs.query for *_, outputs in trace if 'query' in outputs]
    # None of the generated queries exceeds 100 characters in length)
    if max([len(h) for h in hops]) > 100: 
        return False
        
    # None of the generated queries is roughly repeated 
    # (i.e., none is within 0.8 or higher F1 score of earlier queries).
    if any(dspy.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8) for idx in range(2, len(hops))): 
        return False
        
    return True

## Compile (train) the pipeline

In [ ]:
%%time

from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)

compiled_baleen = teleprompter.compile(SimplifiedBaleen(), 
                                       teacher=SimplifiedBaleen(passages_per_hop=2), 
                                       trainset=trainset)

In [32]:
# Ask any question you like to this simple RAG program.
my_question = "How many storeys are in the castle that David Gregory inherited?"

# Get the prediction from uncompiled Baleen
uncompiled_baleen = SimplifiedBaleen()
pred = uncompiled_baleen(my_question)

# Print the contexts and the answer.
print(f"Question: {my_question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Retrieved Contexts (truncated): {[c[:200] + '...' for c in pred.context]}")

Question: How many storeys are in the castle that David Gregory inherited?
Predicted Answer: It is unclear how many storeys the castle that David Gregory inherited had.
Retrieved Contexts (truncated): ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'Delnadamph Lodge | Delnadamph Lodge is located on the Balmoral Estate about eight miles north of the castle. The lodge and its estate lands were bought by Queen Elizabeth II for a figure believed to b...', 'Aydon | Aydon Castle is a fortified manor house and is a Scheduled Ancient Monument and a Grade I listed building. The manor house was built by Robert de Reymes, a wealthy Suffolk merchant, starting i...', 'Floors Castle | Floors Castle, in Roxburghshire, south-east Scotland, is the seat of the Duke of Roxburghe. Despite its name it is a country house rather than a fortress. It wa

In [39]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(dspy.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(dspy.evaluate.normalize_text, [c.split(' | ')[0] for c in pred.context]))

    return gold_titles.issubset(found_titles)

## Evaluate the compiled and uncompiled Baleen pipeline

In [ ]:
from dspy.evaluate.evaluate import Evaluate
# Set up the `evaluate_on_hotpotqa` function.
evaluate_on_hotpotqa = Evaluate(devset=devset, 
                                num_threads=1, 
                                display_progress=True, 
                                display_table=5)

In [41]:
# Evaluate the uncompiled Baleen pipeline
uncompiled_baleen_retrieval_score = evaluate_on_hotpotqa(uncompiled_baleen, 
                                                         metric=gold_passages_retrieved, 
                                                         display=False)
print(f"## Retrieval Score for uncompiled Baleen: {uncompiled_baleen_retrieval_score}")

## Retrieval Score for uncompiled Baleen: 40.0


/Users/ssankar/miniconda3/envs/rag/lib/python3.12/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


In [43]:
# Evaluate the compiled Baleen pipeline
compiled_baleen_retrieval_score = evaluate_on_hotpotqa(compiled_baleen, 
                                                       metric=gold_passages_retrieved,
                                                      display=False)
print(f"## Retrieval Score for compiled Baleen: {compiled_baleen_retrieval_score}")

Error for example in dev set: 		 unsupported operand type(s) for +=: 'int' and 'NoneType'
## Retrieval Score for compiled Baleen: 30.0


/Users/ssankar/miniconda3/envs/rag/lib/python3.12/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


## Save the compiled pipeline, if needed 

In [ ]:
compiled_baleen.save("./dspy_models")
compiled_baleen = SimplifiedBaleen().load("./dspy_models")

### Happy DsPying...
Hope that was helpful. Thank you!